In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
from collections import Counter
import mpmath
from fractions import Fraction
from math import floor
import pickle
import sympy as sp
from scipy.optimize import linprog

Here, we always offer the products with the lowest preference weights.

In [ ]:
# compute E[T_{l1,..,lN}] given the assortment offered at the first step
def T(l, S, list_v, k, memory, assortment):
    """Compute the total expected number of customer required to clear the stock,
    for a policy who offers first the assortment S until a product is selected by a customer, then it follows the optimal policy. 

    Args:
        l (list of integers): the current stock
        S (set): the first assortment offered to the customers, until a product is selected by a customer
        list_v (list): list of the preference weights, without the no-purchase
        k (int): cardinality constraint
        memory (dictionary): dictionary, where the keys are the states of the stock
        assortment (dictionary): dictionary, where the keys are the states of the stock, and the values are the optimal assortment correponding to that stock.

    Returns:
        float: the total expected time
    """
    I_N = np.identity(len(list_v))
    T = Fraction(1,1)
    V_S = Fraction(0,1)
    for i in S :
        V_S = V_S + list_v[i]
        if tuple(l - I_N[i]) in memory :
            Ti = memory[tuple(l - I_N[i])]
        else :
            Ti = f_ordered(l - I_N[i],k, list_v, memory, assortment)[0]
        T = T + list_v[i] *Ti
    return T/V_S + Fraction(1,1)

# compute the feasible assortments
def set_assortments_ordered(l,k):
    """Generate a list of the subsets of elements, of cardinality at most k

    Args:
        elements (list): list of the products which are still in stock
        k (int): cardinality constraint

    Returns:
        list: list of the subsets of elements, of cardinality at most k
    """
    N_assortment = [i for i in range(len(l)) if l[i] > 0]
    if len(N_assortment) > k:
        return N_assortment[:k]
    return N_assortment

# compute the assortment corresponding to the minimum, and the value of the minimum
def f_ordered(l, k, list_v, memory, assortment) :
    """Return the value, where the assortment is ordered by the preference weights, and of cardinality at most k.

    Args:
        l (list of integers): the current stock
        k (int): cardinality constraint
        list_v (list): list of the preference weights, without the no-purchase
        memory (dictionary): dictionary, where the keys are the states of the stock
        assortment (dictionary): dictionary, where the keys are the states of the stock, and the values are the offered assortment correponding to that stock

    Returns:
        float or Fraction: optimal value
        list : offered assortment
        dictionary : dictionary, where the keys are the states of the stock
        dictionary: dictionary, where the keys are the states of the stock, and the values are the offered assortment correponding to that stock

    """
    if tuple(l) in memory :
        return memory[tuple(l)], assortment[tuple(l)], memory, assortment
    S = set_assortments_ordered(l,k)
    T_s = T(l, S, list_v, k, memory, assortment)
    assortment[tuple(l)] = S
    memory[tuple(l)] = T_s
    return T_s, S, memory, assortment

def solution_ordered(l,k, list_v, memory, assortment):
    """Return the dictionaries corresponding to the optimal solution

    Args:
        l (list of integers): the current stock
        k (int): cardinality constraint
        list_v (list): list of the preference weights, without the no-purchase
        memory (dictionary): dictionary, where the keys are the states of the stock
        assortment (dictionary): dictionary, where the keys are the states of the stock, and the values are the optimal assortment correponding to that stock

    Returns:
        dictionary : dictionary, where the keys are the states of the stock
        dictionary: dictionary, where the keys are the states of the stock, and the values are the optimal assortment correponding to that stock.
    """
    N = len(l)
    if tuple([0 for i in range(N)]) not in memory : #initialization of the dictionaries
        memory[tuple([0 for i in range(N)])] = Fraction(0,1)
        assortment[tuple([0 for i in range(N)])] = []
    if tuple(l) in memory:
        return memory, assortment
    if np.sum(l) == 0 :
        return Fraction(0,1),[]
    return f_ordered(l,k,list_v, memory, assortment)[2:4]

In [ ]:
memory = {}
assortment = {}
list_v = [Fraction(1,1), Fraction(4,1), Fraction(5,1)]

In [ ]:
list_l = [10,10,10]
memory, assortment = solution_ordered(list_l,2,list_v, memory, assortment)